In [11]:
import pandas as pd
import os

code_analysis_generated_path = "../../generated/code-analysis"
if not os.path.exists(code_analysis_generated_path):
    os.makedirs(code_analysis_generated_path)

reports_path = "../../generated/reports-no-bug"

data = []

users = os.listdir(reports_path)
if len(users) == 0:
    raise FileNotFoundError(f"Le dossier {reports_path} est vide. Copier les uploads dans ce dossier avant d'exécuter le notebook.")

for user in users:
    uploads_path = os.path.join(reports_path, user)
    uploads = os.listdir(uploads_path)
    for upload in uploads:
        path = os.path.join(uploads_path, upload)

        jacoco_path = os.path.join(path, "JaCoCo", "jacoco.csv")
        jacoco_valid = os.path.exists(jacoco_path)

        pitest_path = os.path.join(path, "Pitest", "mutations.xml")
        pitest_valid = os.path.exists(pitest_path)

        data.append([user, upload, jacoco_path, jacoco_valid, pitest_path, pitest_valid])

main_df = pd.DataFrame(data, columns=['user', 'timestamp', 'jacoco', 'jacoco_valid', 'pitest', 'pitest_valid'])
main_df['timestamp'] = pd.to_datetime(main_df['timestamp'], format='%Y-%m-%dT%H-%M-%S.%fZ', utc=True).dt.tz_convert("Europe/Brussels")

# Count number of valid projects per user
user_stats = main_df.groupby('user').agg({
    'jacoco_valid': 'sum',
    'pitest_valid': 'sum'
}).reset_index()

# Count users with 0, 1, or 2 valid projects
jacoco_counts = user_stats['jacoco_valid'].value_counts().sort_index()
pitest_counts = user_stats['pitest_valid'].value_counts().sort_index()

total_users = len(user_stats)

# Create DataFrame with counts and percentages
df = pd.DataFrame({
    'metric': ['jacoco', 'jacoco', 'jacoco', 'pitest', 'pitest', 'pitest'],
    'valid_projects': [0, 1, 2] * 2,
    'count': list(jacoco_counts) + list(pitest_counts),
    'percentage': list((jacoco_counts / total_users) * 100) + list((pitest_counts / total_users) * 100)
})

df


,metric,valid_projects,count,percentage
0,jacoco,0,4,13.793103
1,jacoco,1,9,31.034483
2,jacoco,2,16,55.172414
3,pitest,0,4,13.793103
4,pitest,1,9,31.034483
5,pitest,2,16,55.172414
